In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import Dataset, DataLoader, sampler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.functional as F
from torch import optim
from skimage import io, transform
from timeit import default_timer as timer

from PIL import Image

%matplotlib inline 

In [ ]:
df = pd.read_csv('../input/train.csv')
df.head()

In [ ]:
#Dataset class

class ImageDataset(Dataset):
    

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with labels.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame['Id'][idx])         # getting path of image
        image = Image.open(img_name).convert('RGB')                                # reading image and converting to rgb if it is grayscale
        label = np.array(self.data_frame['Category'][idx])                         # reading label of the image
        
        if self.transform:            
            image = self.transform(image)                                          # applying transforms, if any
        
        sample = (image, label)        
        return sample

In [ ]:
# Image transformations
image_transforms = {
    # Train uses data augmentation along with normalization.
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees = 25),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        #transforms.RandomVerticalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation but uses the same normalization.
    'val':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Test does not use augmentation but uses the same normalization.
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
# making the datasets which will be passed to dataloader.

trainset = ImageDataset(csv_file = '../input/train.csv', root_dir = '../input/data/data/', transform=image_transforms['train'])
#validset = ImageDataset(csv_file = '../input/train.csv', root_dir = '../input/data/data/', transform=image_transforms['val'])
testset = ImageDataset(csv_file = '../input/sample_sub.csv', root_dir = '../input/data/data/', transform=image_transforms['test'])

In [ ]:
valid_size = 0.20

# obtain training indices that will be used for validation
num_train = len(trainset)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)


In [ ]:
# data loader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=15, sampler = train_sampler, num_workers=0)
valid_loader = torch.utils.data.DataLoader(trainset, batch_size=15, sampler=valid_sampler, num_workers=0)
test_loader = torch.utils.data.DataLoader(testset, batch_size=15, shuffle=False, num_workers=0)

In [ ]:
#Checking training sample size and label
for i in range(len(trainset)):
    sample = trainset[i]
    print(i, sample[0].size(), " | Label: ", sample[1])
    if i == 10:
        break

In [ ]:
# Getting the shape of each batch
trainiter = iter(train_loader)
features, labels = next(trainiter)
features.shape, labels.shape

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(15):                                             #Change the range according to your batch-size
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))

In [ ]:
# check if GPU available or not
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
model = models.resnet152(pretrained = True)

In [ ]:
model

In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [ ]:
model.fc =  nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024,eps=1e-05, momentum=0.1, affine=True),
            nn.Dropout(0.35),
            nn.Linear(1024, 67),
#             nn.Linear(1024, 512),
#             nn.ReLU(),
#             nn.BatchNorm1d(512,eps=1e-05, momentum=0.1, affine=True),
#             nn.Dropout(0.35),
#             nn.Linear(512, 67),
            nn.LogSoftmax(dim=1)
            
)

In [ ]:
for name, child in model.named_children():
    if name in ['layer3','layer4','fc']:
        print(name + ' is unfrozen')
        for param in child.parameters():
            param.requires_grad = True
    else:
        print(name + ' is frozen')
        for param in child.parameters():
            param.requires_grad = False

In [ ]:
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0006, momentum=0.9, weight_decay=1e-5)

In [ ]:
model

In [ ]:

criterion = nn.CrossEntropyLoss()


#optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0008, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [ ]:
if train_on_gpu:
    model.cuda()
    print("moved")

In [ ]:
#Training the model and saving checkpoints of best performances. That is lower validation loss and higher accuracy
epochs = 50
valid_loss_min = np.Inf
import time
for epoch in range(epochs):
    
    
    start = time.time()
    
    scheduler.step()
    model.train()
    
    train_loss = 0.0
    valid_loss = 0.0
    
    for inputs, labels in train_loader:
        
        
       
        # Move input and label tensors to the default device
        inputs, labels = inputs.cuda(), labels.cuda()
        
        optimizer.zero_grad()
        
        logps = model(inputs)
#         _, preds = torch.max(logps, 1)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
       
    model.eval()
    
    with torch.no_grad():
        accuracy = 0
        for inputs, labels in valid_loader:
            
            inputs, labels = inputs.cuda(), labels.cuda()
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
            valid_loss += batch_loss.item()
            
            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
    
                    
            
            
    # calculate average losses
    train_loss = train_loss/len(train_loader)
    valid_loss = valid_loss/len(valid_loader)
    valid_accuracy = accuracy/len(valid_loader) 
      
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Accuracy: {:.6f}'.format(
        epoch + 1, train_loss, valid_loss, valid_accuracy))
            
    
    
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        torch.save(model.state_dict(), 'model_resnet152_new.pt')
        valid_loss_min = valid_loss        
       
    print(f"Time per epoch: {(time.time() - start):.3f} seconds")

In [ ]:
model.load_state_dict(torch.load('model_resnet152_new.pt'))

In [ ]:
# Reading sample_submission file to get the test image names
submission = pd.read_csv('../input/sample_sub.csv')
submission.head()

In [ ]:
# iterate over test data to make predictions
predictions = []
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    _, pred = torch.max(output, 1)
    for i in range(len(pred)):
        predictions.append(int(pred[i]))
        

submission['Category'] = predictions       #Attaching predictions to submission file

In [ ]:
#saving submission file
submission.to_csv('submission.csv', index=False, encoding='utf-8')